In [35]:
import sys
import pprint
import datetime
from tlslite.utils import keyfactory
import pymongo
import pprint
import math
import pandas as pd
import numpy as np
from plotly import tools
import plotly.graph_objs as go
import plotly.offline as py
import os
import requests
import time
from bs4 import BeautifulSoup

py.init_notebook_mode(connected=True)

In [5]:
client = pymongo.MongoClient('18.8.8.120')
cydb = client.cy_useract
issues = pd.DataFrame(list(cydb.useract_info.find()))
del issues['_id']
len(issues)
valid_data = issues.loc[issues['IMEI'].apply(lambda x:len(x.split(';')) == 2)]

client = pymongo.MongoClient('18.8.8.209')
cydb = client.cy
#topapps = pd.DataFrame(list(cydb.topapps.find(projection={'name': 1, 'package_name':1})))
#del topapps['_id']
#cy_packages = pd.DataFrame(list(cydb.cy_packages.find()))
#del cy_packages['_id']

In [13]:
len(issues)

573451

In [31]:
#获取包对应的应用名
total_apps = pd.DataFrame(list(cydb.package2name.find(projection={'_id':0, 'name': 1, 'package_name':1})))
total_apps_names = total_apps.set_index('package_name').to_dict()['name']

In [6]:
#获取3月，4月的数据
last_2month = datetime.datetime(2018, 4, 18).timestamp()
last_month = datetime.datetime(2018, 5, 18).timestamp()
cur_month = datetime.datetime(2018, 6, 18).timestamp()

df_last_month = valid_data[(valid_data['wallTime'] >= last_2month) & (valid_data['wallTime'] < last_month)]
df_cur_month = valid_data[(valid_data['wallTime'] >= last_month) & (valid_data['wallTime'] < cur_month)]

In [7]:
###获取包名
def get_name_from_google(package_name):
    name = 'Not Found'
    url = "https://play.google.com/store/apps/details"
    querystring = {"id":package_name}

    headers = {
        'accept': "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
        'accept-encoding': "gzip, deflate, br",
        'accept-language': "zh-CN,zh;q=0.9,en;q=0.8,zh-TW;q=0.7",
        'upgrade-insecure-requests': "1",
        'cookie': 'SID=DAaVKqDYVfUDOvDttkKw4L9_fFoj3k4amZyPFCJv2-JrhZoD2gLvVPLqDS9ipsTYjwXWmw.; HSID=Az021_6ePZogKY8cM; SSID=ACpzbNvy8_x8Vnybk; APISID=7S1tb3ZsgGv_bYvR/ACjOTyZPnt4umXRCq; SAPISID=xjIFc_YmAa63ey27/AmEbiR4E6aujvkXPS; NID=130=ab4LxWGm30bUFH_KitKmxCkBbExye2K9qU5VTdygb49TBNA1a8pN4jyad6LYrAPvhIpVO6_iNDLTOJuWxrwzgPd3qCm6uC6LFx8txajCZp-xDO7tdQIGf_eQYLM_BVGPdAQ47Oa2WUX_9DshsR3iJItArmKVShCwPvA2HCFO6XLBDaHjyRpXAhPrYKZfGIQ0PVYRkT8hBjzDCtPM4VcHKbSmMi0SCQ1dVGJDTKbALIYYCBwODH_mQsStZA4FHzSzT71NG-xMPONgrzmi9h6Nm1Xo; OGPC=19005936-1:; S=billing-ui-v3=H2iK8QW8I6T65a977LlFh58Un3NygpND:billing-ui-v3-efe=H2iK8QW8I6T65a977LlFh58Un3NygpND; SIDCC=AEfoLebYzi5VJ4REqjhIorMwY7WG7kadx86R29vcphaujmkiGT8I2tDqnP5_VwSYG_kLoRECJg; 1P_JAR=2018-5-18-7',
        'user-agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/537.36  \
        (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36",
        'x-client-data': "CKe1yQEIibbJAQiltskBCMG2yQEIq5jKAQipncoBCN6eygEIqKPKARiSo8oB",
        'cache-control': "no-cache",
        }

    response = requests.request("GET", url, headers=headers, params=querystring)

    if response.status_code == 200:
        soup_app = BeautifulSoup(response.text, 'lxml')
        href_all = soup_app.find_all('h1')
        name = href_all[0].span.string
        
    return name

In [36]:
def get_bar_table_data(cur_month, last_month, rate=True, need_other=True):
    top30_click = cur_month[-30:]
    others = cur_month.sum() - top30_click.sum()
    
    if need_other:
        top31_click = top30_click.append(pd.Series({'Others':others})).sort_values()
    else:
        top31_click = top30_click
    appclick_diff = cur_month.sub(last_month, fill_value=0)[list(top30_click.index)]

    package_name_diff = list(appclick_diff.index)
    package_name_diff.reverse()

    app_name_diff = []
    for package in package_name_diff:
        if package in total_apps_names.keys():
            if total_apps_names[package] != 'Not Found':
                app_name_diff.append(total_apps_names[package])
            else:
                app_name_diff.append(package)
        else:
            print('1 get package name from google:' + package)
            name = get_name_from_google(package)
            total_apps_names[package] = name
            if name == 'Not Found':
                app_name_diff.append(package)
            else:
                app_name_diff.append(name)
                cydb.package2name.insert_one({'name':name, 'package_name':package})
            time.sleep(2)
    
    value_diff = list(appclick_diff.values)
    value_diff.reverse()
    
    packages_name = []
    package_list = list(total_apps['package_name'])
    for package in top31_click.index:
        if package in total_apps_names.keys():
            if total_apps_names[package] != 'Not Found':
                packages_name.append(total_apps_names[package])
            else:
                packages_name.append(package)
        else:
            print('3 get package name from google:' + package)
            name = get_name_from_google(package)
            total_apps_names[package] = name
            if name == 'Not Found':
                packages_name.append(package)
            else:
                packages_name.append(name)
                cydb.package2name.insert_one({'name':name, 'package_name':package})
            time.sleep(2)
    
    bar_data = {}
    bar_data['y'] = packages_name
    bar_data['x'] = top31_click.values
    if rate:
        bar_data['text'] = top31_click.apply(lambda x:'{:.2f}%'.format(x*100)).values
        table_data = [app_name_diff, ['{:.2f}%'.format(x*100) for x in value_diff]]
    else:
        bar_data['text'] = top31_click.apply(lambda x:round(x,1)).values
        table_data = [app_name_diff, [round(x,1) for x in value_diff]]
    
    
    return bar_data, table_data

In [37]:
def draw_bar_table(bar_data, table_data, height, width, title, table_pos={'x':[0.45, 0.9], 'y':[0, 0.8]}):
    table_trace = go.Table(
        domain=table_pos,
        columnwidth = [52]*2,
        columnorder=[0, 1],
        header = dict(height = 25,
                    values = [['<b>名称</b>'],['<b>与上月同比</b>'],],
                    line = dict(color='#000000'),
                    align = ['left'] * 2,
                    font = dict(color=['#FFFFFF'] * 2, size=14),
                    fill = dict(color='#2d5b6c')),
        cells = dict(values = table_data,
                    line = dict(color='#000000'),
                    align = ['left'] * 2,
                    font = dict(color=['#FFFFFF'] * 2, size=12),
                    #format = [None]*4 + [',.2f'],
                    #prefix = [None] * 2 + ['$', u'\u20BF'],
                    #suffix=[None] * 4,
                    height = 25,
                    fill = dict(color=['#68a4b9', '#526372']))
    )

    trace = go.Bar(y=bar_data['y'], x=bar_data['x'], orientation = 'h',
                  text = bar_data['text'],textposition = 'outside',)

    layout = go.Layout(
        title=title,
        height=height,
        width=width,
        margin=dict(l=300,r=0),
        yaxis = dict(ticklen=2)
    )

    fig = go.Figure(data=[trace,table_trace],layout=layout)
    py.iplot(fig)


In [38]:
imeis_cur_month = len(df_cur_month['IMEI'].unique())
imeis_last_month = len(df_last_month['IMEI'].unique())

total_appclick_cur_month = df_cur_month[df_cur_month['name'] == 'appclick']
total_appclick_last_month = df_last_month[df_last_month['name'] == 'appclick']

In [41]:
appclick_cur_month = total_appclick_cur_month.groupby(['attrs_packagename']).size().sort_values().truediv(len(total_appclick_cur_month))
appclick_last_month = total_appclick_last_month.groupby(['attrs_packagename']).size().sort_values().truediv(len(total_appclick_last_month))

title='点击次数排名<br>总点击数:{}  设备数:{}'.format(len(total_appclick_cur_month), imeis_cur_month)
bar_data, table_data = get_bar_table_data(appclick_cur_month, appclick_last_month)

1 get package name from google:com.android.chrome
1 get package name from google:com.nemo.vidmate
1 get package name from google:com.truecaller
1 get package name from google:com.katecca.screenofflock
1 get package name from google:com.google.android.googlequicksearchbox
1 get package name from google:com.linecorp.b612.android
3 get package name from google:Others


In [44]:
draw_bar_table(bar_data, table_data, height=1300, width=1300, title=title)

In [16]:
get_name_from_google('com.facebook.orca')

'Messenger – Text and Video Chat for Free'

In [46]:
click_by_phone_cur_month = total_appclick_cur_month.groupby(['attrs_packagename']).size().sort_values().truediv(imeis_cur_month)
click_by_phone_cur_month = click_by_phone_cur_month.apply(lambda x: round(x,1))
click_by_phone_last_month = total_appclick_last_month.groupby(['attrs_packagename']).size().sort_values().truediv(imeis_last_month)
click_by_phone_last_month = click_by_phone_last_month.apply(lambda x: round(x,1))

title='人均点击排名<br>总点击数:{}  设备数:{}'.format(len(total_appclick_cur_month), imeis_cur_month)
bar_data, table_data = get_bar_table_data(click_by_phone_cur_month, click_by_phone_last_month, False)
draw_bar_table(bar_data, table_data, height=1300, width=1300, title=title)

In [47]:
def install_rate(cur_month, last_month, label,title,height=1200,width=1200,by_imei=True):
    data_cur = cur_month[cur_month.name == label].groupby(['IMEI', 'attrs_packagename']).size()
    data_last = last_month[last_month.name == label].groupby(['IMEI', 'attrs_packagename']).size()
    
    package_info_cur = data_cur.reset_index()
    package_info_last = data_last.reset_index()
    
    imeis_cur = len(cur_month['IMEI'].unique())
    imeis_last = len(last_month['IMEI'].unique())
    total_cur = package_info_cur.groupby('attrs_packagename').size().sort_values().sum()
    total_last = package_info_last.groupby('attrs_packagename').size().sort_values().sum()
    
    if by_imei:
        divisor_cur = imeis_cur
        divisor_last = imeis_last
    else:
        divisor_cur = total_cur
        divisor_last = total_last
        
    package_rate_cur = package_info_cur.groupby('attrs_packagename').size().sort_values()/divisor_cur
    package_rate_last = package_info_last.groupby('attrs_packagename').size().sort_values()/divisor_last
    
    bar_data, table_data = get_bar_table_data(package_rate_cur, package_rate_last, rate=True, need_other=False)
    title= title + '<br>总次数:{}  总设备数:{}'.format(str(int(total_cur)), imeis_cur)
 
    draw_bar_table(bar_data, table_data, height, width, title)

In [48]:
install_rate(df_cur_month, df_last_month, 'appinstall', '安装应用排行', height=1300,width=1300,by_imei=False)

1 get package name from google:com.snaptube.premium
1 get package name from google:com.domobile.applock
1 get package name from google:com.venticake.retrica
1 get package name from google:com.parfield.prayers.lite
1 get package name from google:com.etisalat
1 get package name from google:com.olxmena.horizontal
1 get package name from google:com.firsttouchgames.story
1 get package name from google:net.mbc.shahid
1 get package name from google:com.springwalk.mediaconverter


In [49]:
install_rate(df_cur_month, df_last_month, 'appinstall', '安装应用覆盖率排行', height=1300,width=1300)

In [51]:
install_rate(df_cur_month, df_last_month,'appremove', '卸载应用排行',height=1400,width=1300)

### Camera 应用 

In [52]:
chinese_name_photos = {'photo':'普通拍照', 'panorama':'全景', 'facebeauty':'美颜', 'hdr':'逆光', 'nightvideo':'夜景',
                'selfiepanorama':'全景自拍', 'polaroid':'心情照片','gifshot':'GIF动画','profession':'高级', 'arcfilter':'滤镜'}
chinese_name_videos = {'video':'普通摄像','slowmotion':'慢动作摄影', 'timelapse':'延时摄影'}

In [53]:
def draw_camera(cur, last, height, width, title, table_pos={'x':[0.45, 0.9], 'y':[0, 0.86]}):
    cur_rate = cur.apply(lambda x:x/cur.sum())
    last_rate = last.apply(lambda x:x/last.sum())
    diff_data = cur_rate.sub(last_rate, fill_value=0)
    table_data = [diff_data.index, ['{:.2f}%'.format(x*100) for x in diff_data]]
    bar_data = {'y':cur_rate.index, 'x':cur_rate.values,
                'text':cur_rate.apply(lambda x:'{:.1f}%'.format(x*100)).values}
    draw_bar_table(bar_data, table_data, height, width, title, table_pos)

In [54]:
def get_camera_data(df):
    takephoto_data_cur = df[df['name'] == 'TakePhoto']
    video_data_cur = df[df['name'] == 'Video']

    total_camera_cur = len(takephoto_data_cur)
    total_video_cur = len(video_data_cur)
    total_used_cur = (len(takephoto_data_cur) + len(video_data_cur))

    capturemode_cur = takephoto_data_cur.groupby(['attrs_Capturemode']).size().sort_values()

    capture_dict_cur = {}
    total_camera_dict_cur = {}
    for p in chinese_name_photos:
        if p in capturemode_cur.index:
            capture_dict_cur[chinese_name_photos[p]] = capturemode_cur[p]
            total_camera_dict_cur[chinese_name_photos[p]] = capturemode_cur[p]
        else:
            total_camera_dict_cur[chinese_name_photos[p]] = 0
            capture_dict_cur[chinese_name_photos[p]] = 0

    capture_chinese_data = pd.Series(capture_dict_cur).sort_values()

    video_mode = video_data_cur.groupby(['attrs_Videomode']).size().sort_values()
    video_dict_cur = {}
    for p in chinese_name_videos:
        if p in video_mode.index:
            video_dict_cur[chinese_name_videos[p]] = video_mode[p]
            total_camera_dict_cur[chinese_name_videos[p]] = video_mode[p]
        else:
            capture_dict_cur[chinese_name_videos[p]] = 0
            total_camera_dict_cur[chinese_name_videos[p]] = 0

    video_chinese_data = pd.Series(video_dict_cur).sort_values()
    total_chinese_data = pd.Series(total_camera_dict_cur).sort_values()
    
    return [capture_chinese_data,video_chinese_data,total_chinese_data]

In [55]:
cur_camera = get_camera_data(df_cur_month)
last_camera = get_camera_data(df_last_month)

In [56]:
draw_camera(cur_camera[0], last_camera[0], 550, 1200, '拍照使用排名<br>总使用次数:{}'.format(cur_camera[0].sum()))

In [57]:
draw_camera(cur_camera[1], last_camera[1], 400, 1200, '录像使用排名<br>总使用次数:{}'.format(cur_camera[1].sum()),
           table_pos={'x':[0.45, 0.9], 'y':[0, 0.6]})

In [58]:
draw_camera(cur_camera[2], last_camera[2], 600, 1200, '相机使用排名<br>总使用次数:{}'.format(cur_camera[2].sum()))

### 销量统计

In [ ]:
#valid_data.groupby(['IMEI', 'wallTime']).size()

In [59]:
imeis = len(valid_data['IMEI'].unique())

In [60]:
d518 = datetime.datetime(2018, 1, 1).timestamp()

In [61]:
vvdata = valid_data[valid_data['registerTime'] >= d518]
#imeis = len(vvdata['IMEI'].unique())
register = vvdata.groupby(['IMEI', 'registerTime']).size()
register_data = {}

for imei, group in register.reset_index().groupby(['IMEI']):
    register_date = datetime.datetime.fromtimestamp(group.iloc[-1]['registerTime'])
    register_data[imei] = register_date

In [62]:
psd = pd.DataFrame.from_dict(register_data, orient='index')

In [63]:
register_count = psd.groupby([0]).size()

In [64]:
cc = register_count.resample('D').sum().fillna(0)

In [65]:
paper_bgcolor='#242640'
plot_bgcolor='#242640'
paper_bgcolor='#FFFFFF'
plot_bgcolor='#FFFFFF'
text_color = '#000000'
x_color = '#000000'
y_color = '#000000'
title_color = '#000000'
legend_color = '#000000'

In [66]:
cc.index.strftime('%m-%d')

array(['01-01', '01-02', '01-03', '01-04', '01-05', '01-06', '01-07',
       '01-08', '01-09', '01-10', '01-11', '01-12', '01-13', '01-14',
       '01-15', '01-16', '01-17', '01-18', '01-19', '01-20', '01-21',
       '01-22', '01-23', '01-24', '01-25', '01-26', '01-27', '01-28',
       '01-29', '01-30', '01-31', '02-01', '02-02', '02-03', '02-04',
       '02-05', '02-06', '02-07', '02-08', '02-09', '02-10', '02-11',
       '02-12', '02-13', '02-14', '02-15', '02-16', '02-17', '02-18',
       '02-19', '02-20', '02-21', '02-22', '02-23', '02-24', '02-25',
       '02-26', '02-27', '02-28', '03-01', '03-02', '03-03', '03-04',
       '03-05', '03-06', '03-07', '03-08', '03-09', '03-10', '03-11',
       '03-12', '03-13', '03-14', '03-15', '03-16', '03-17', '03-18',
       '03-19', '03-20', '03-21', '03-22', '03-23', '03-24', '03-25',
       '03-26', '03-27', '03-28', '03-29', '03-30', '03-31', '04-01',
       '04-02', '04-03', '04-04', '04-05', '04-06', '04-07', '04-08',
       '04-09', '04-

In [67]:
cc.index.strftime('%y-%m-%d')

array(['18-01-01', '18-01-02', '18-01-03', '18-01-04', '18-01-05',
       '18-01-06', '18-01-07', '18-01-08', '18-01-09', '18-01-10',
       '18-01-11', '18-01-12', '18-01-13', '18-01-14', '18-01-15',
       '18-01-16', '18-01-17', '18-01-18', '18-01-19', '18-01-20',
       '18-01-21', '18-01-22', '18-01-23', '18-01-24', '18-01-25',
       '18-01-26', '18-01-27', '18-01-28', '18-01-29', '18-01-30',
       '18-01-31', '18-02-01', '18-02-02', '18-02-03', '18-02-04',
       '18-02-05', '18-02-06', '18-02-07', '18-02-08', '18-02-09',
       '18-02-10', '18-02-11', '18-02-12', '18-02-13', '18-02-14',
       '18-02-15', '18-02-16', '18-02-17', '18-02-18', '18-02-19',
       '18-02-20', '18-02-21', '18-02-22', '18-02-23', '18-02-24',
       '18-02-25', '18-02-26', '18-02-27', '18-02-28', '18-03-01',
       '18-03-02', '18-03-03', '18-03-04', '18-03-05', '18-03-06',
       '18-03-07', '18-03-08', '18-03-09', '18-03-10', '18-03-11',
       '18-03-12', '18-03-13', '18-03-14', '18-03-15', '18-03-

In [68]:
cc.values

array([ 195.,    3.,    0.,    1.,    2.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    1.,    0.,    0.,
          1.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    1.,    1.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    2.,
          0.,    0.,    0.,    1.,    0.,    0.,    0.,    1.,    1.,
          0.,    1.,    0.,    0.,    0.,   10.,   10.,    4.,    1.,
          5.,    2.,    3.,    4.,    0.,    0.,    3.,    2.,    0.,
          6.,    3.,    3.,    5.,    6.,    6.,    4.,    2.,    8.,
          5.,    5.,    6.,    7.,    6.,    0.,    8.,   11.,    7.,
          7.,    9.,    3.,    2.,    4.,   13.,    5.,    5.,    8.,
          9.,    8.,    2.,    2.,    4.,    2.,   10.,    5.,    3.,
          4.,    2.,

In [69]:
bandxaxis = go.XAxis(
        type='date',
        #range=[(created_by_day.index[start] - 1).strftime('%y-%m-%d'),
        #       (created_by_day.index[-1] + 1).strftime('%y-%m-%d'),],
        ticks="outside", 
        showticklabels=True,
        ticktext=list(cc.index.strftime('%m-%d')),
        tickvals=list(cc.index.strftime('%y-%m-%d')),
        tickfont=dict(size=11),
        #tickfont=dict(size=11,color='#B9B9C3'),
        ticklen=10,showgrid=True,
        #color='#FFFFFF'
    )

In [70]:
data=[go.Bar(
        x = cc.index,
        y = cc.values,
        text=cc.values, 
        textposition='outside',)
        #marker=dict(color='#7D858C')), 
]


In [72]:
layout = go.Layout(
    title='销量趋势图<br>总数:' + str(imeis),
    titlefont=dict(color=title_color,size=25),
    height=1200,
    width=2000,
    bargap=0.1,
    xaxis = bandxaxis,
    yaxis = dict(ticklen=10),
    #bargroupgap=0.05,
    #paper_bgcolor=paper_bgcolor,
    #plot_bgcolor=plot_bgcolor,
    legend=dict(
        x=0.1,
        font=dict(size=16),
    )
)

fig = go.Figure(data=data,layout=layout)
py.iplot(fig)

In [ ]:
trace1= go.Scatter(x=[0,0.5,1,2,2.2],y=[1.23,2.5,0.42,3,1])
layout= go.Layout(images= [dict(
        source= "https://lh3.googleusercontent.com/ZZPdzvlpK9r_Df9C3M7j1rNRi7hhHRvPhlklJ3lfi5jk86Jd1s0Y5wcQ1QgbVaAP5Q=s360-rw",
        xref= "paper",
        yref= "paper",
        x= 0,
        y= 1,
        sizex= 0.1,
        sizey= 0.1,
        xanchor= "right",
        yanchor= "top"
      ),dict(
        source= "https://lh3.googleusercontent.com/ZZPdzvlpK9r_Df9C3M7j1rNRi7hhHRvPhlklJ3lfi5jk86Jd1s0Y5wcQ1QgbVaAP5Q=s360-rw",
        xref="x",
        yref= "y",
        x= 1.5,
        y= 2,
        sizex= 1,
        sizey= 1,
        xanchor= "right",
        yanchor= "bottom"
      ),
      dict(
        source= "https://lh3.googleusercontent.com/ZZPdzvlpK9r_Df9C3M7j1rNRi7hhHRvPhlklJ3lfi5jk86Jd1s0Y5wcQ1QgbVaAP5Q=s360-rw",
        xref= "x",
        yref= "y",
        x= 2,
        y= 1,
        sizex= 0.3,
        sizey= 0.5,
        sizing= "stretch",
        opacity= 0.4,
        layer= "below"
      ),
      dict(
        source= "https://lh3.googleusercontent.com/ZZPdzvlpK9r_Df9C3M7j1rNRi7hhHRvPhlklJ3lfi5jk86Jd1s0Y5wcQ1QgbVaAP5Q=s360-rw",
        xref= "x",
        yref= "paper",
        x= 3,
        y= 0,
        sizex= 0.5,
        sizey= 1,
        opacity= 1,
        xanchor= "right",
        yanchor="middle"
      )])
fig=go.Figure(data=[trace1],layout=layout)
py.iplot(fig)